AutoGluonのチュートリアルの1つテーブルデータでの予測を実施
https://autogluon.mxnet.io/tutorials/tabular_prediction/tabular-quickstart.html

本問題は収入が500万円（正確には$50,000）を超えるかどうかを予想する

まずはAutoGluon実行に必要なライブラリを読み込む
TabularPredictionはTable（CSVデータ）を読み込んで学習するモジュール

In [3]:
import autogluon as ag
from autogluon import TabularPrediction as task

データセットをS3から読み込む
データセットの内、先頭の500件を訓練用データセットとして使用する。
ちなみにデータ内の各変数の説明は以下のURLに詳しく書いている
https://archive.ics.uci.edu/ml/datasets/adult

age:年齢
workclass:
fnlwgt:？？
education:学歴
education-num:
marital-status:婚姻状況
occupation:職業
relationship:続柄
race:人種
sex:性別
capital-gain:
capital-loss:
hours-per-week:1週間の労働時間
native-country:出身国
class:年収が500万を超える(>50K)かそれ以下(<=50K)か

In [4]:
train_data = task.Dataset(file_path='https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
train_data = train_data.head(500) # subsample 500 data points for faster demo
print(train_data.head())

Loaded data from: https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv | Columns = 15 / 15 | Rows = 39073 -> 39073


   age   workclass  fnlwgt   education  education-num       marital-status  \
0   25     Private  178478   Bachelors             13        Never-married   
1   23   State-gov   61743     5th-6th              3        Never-married   
2   46     Private  376789     HS-grad              9        Never-married   
3   55           ?  200235     HS-grad              9   Married-civ-spouse   
4   36     Private  224541     7th-8th              4   Married-civ-spouse   

           occupation    relationship    race      sex  capital-gain  \
0        Tech-support       Own-child   White   Female             0   
1    Transport-moving   Not-in-family   White     Male             0   
2       Other-service   Not-in-family   White     Male             0   
3                   ?         Husband   White     Male             0   
4   Handlers-cleaners         Husband   White     Male             0   

   capital-loss  hours-per-week  native-country   class  
0             0              40   United

予測する収入をラベルとして内容の確認を行う。
内容を見るにタイプとしては50k以上か未満の2つしかないことがわかる。

In [5]:
label_column = 'class'
print("Summary of class variable: \n", train_data[label_column].describe())

Summary of class variable: 
 count        500
unique         2
top        <=50K
freq         394
Name: class, dtype: object


fit関数を呼び出し訓練用データセットとラベルを設定し学習をさせる。
ouput_directoryを指定することで学習したモデルを該当のディレクトリにアウトプットすることができる。

In [6]:
dir = 'models/agModels-predictClass' # specifies folder where to store trained models
predictor = task.fit(train_data=train_data, label=label_column, output_directory=dir)

Beginning AutoGluon training ...
AutoGluon will save models to agModels-predictClass/
AutoGluon Version:  0.0.12
Train Data Rows:    500
Train Data Columns: 15
Preprocessing data ...
Here are the 2 unique label values in your data:  [' <=50K', ' >50K']
AutoGluon infers your prediction problem is: binary  (because only two unique label-values observed).
If this is wrong, please specify `problem_type` argument in fit() instead (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])

Selected class <--> label mapping:  class 1 =  >50K, class 0 =  <=50K
Train Data Class Count: 2
Feature Generator processed 500 data points with 14 features
Original Features (raw dtypes):
	int64 features: 6
	object features: 8
Original Features (inferred dtypes):
	int features: 6
	object features: 8
Generated Features (special dtypes):
Final Features (raw dtypes):
	int features: 6
	category features: 8
Final Features:
	int features: 6
	category features: 8
	Data preprocessing and fea

次にS3のデータから検証データセットを作成する。
drop関数を使用することでラベルを削除して訓練データセットと形式を合わす。
個人的に気になるのは検証データはオリジナルとなるS3の全データを使用しているが、この中に訓練用データセットも含まれている。
それって駄目なのでは？？

In [7]:
test_data = task.Dataset(file_path='https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
y_test = test_data[label_column]  # values to predict
# dropで訓練データセットと合わす
test_data_nolab = test_data.drop(labels=[label_column],axis=1) # delete label column to prove we're not cheating
print(test_data.head())
print(test_data_nolab.head())

Loaded data from: https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv | Columns = 15 / 15 | Rows = 9769 -> 9769


   age          workclass  fnlwgt      education  education-num  \
0   31            Private  169085           11th              7   
1   17   Self-emp-not-inc  226203           12th              8   
2   47            Private   54260      Assoc-voc             11   
3   21            Private  176262   Some-college             10   
4   17            Private  241185           12th              8   

        marital-status        occupation relationship    race      sex  \
0   Married-civ-spouse             Sales         Wife   White   Female   
1        Never-married             Sales    Own-child   White     Male   
2   Married-civ-spouse   Exec-managerial      Husband   White     Male   
3        Never-married   Exec-managerial    Own-child   White   Female   
4        Never-married    Prof-specialty    Own-child   White     Male   

   capital-gain  capital-loss  hours-per-week  native-country   class  
0             0             0              20   United-States   <=50K  
1       

学習し作成したモデルはpredictorという変数に格納したが、ディレクトリに出力したものからロードすることも可能。
ロードしたモデルからpredict関数を実行することで検証用データから予想を行う。
またevaluate_predictionsで予想した結果と実際の値から評価をする。

In [8]:
predictor = task.load(dir) # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(test_data_nolab)
print("Predictions:  ", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: accuracy on test data: 0.8117514586958747
Evaluations on test data:
{
    "accuracy": 0.8117514586958747,
    "accuracy_score": 0.8117514586958747,
    "balanced_accuracy_score": 0.6263545355685329,
    "matthews_corrcoef": 0.39455555206800325,
    "f1_score": 0.8117514586958747
}


Predictions:   [' <=50K' ' <=50K' ' <=50K' ... ' <=50K' ' <=50K' ' <=50K']


Detailed (per-class) classification report:
{
    " <=50K": {
        "precision": 0.8124721603563474,
        "recall": 0.9791974231646758,
        "f1-score": 0.8880774146430527,
        "support": 7451
    },
    " >50K": {
        "precision": 0.8035487959442332,
        "recall": 0.27351164797238997,
        "f1-score": 0.4081107177341487,
        "support": 2318
    },
    "accuracy": 0.8117514586958747,
    "macro avg": {
        "precision": 0.8080104781502904,
        "recall": 0.6263545355685329,
        "f1-score": 0.6480940661886007,
        "support": 9769
    },
    "weighted avg": {
        "precision": 0.8103548137796988,
        "recall": 0.8117514586958747,
        "f1-score": 0.7741903429433045,
        "support": 9769
    }
}


fit_summary関数を使用することで学習過程の概要が表示できる

In [10]:
print(predictor.fit_summary())

*** Summary of fit() ***
Estimated performance of each model:
                         model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer
0      weighted_ensemble_k0_l1       0.88       0.438152  10.342503                0.001542           0.599641            1       True
1           LightGBMClassifier       0.86       0.015446   0.398098                0.015446           0.398098            0       True
2          NeuralNetClassifier       0.86       0.046011   7.275923                0.046011           7.275923            0       True
3           CatboostClassifier       0.85       0.014026   0.876040                0.014026           0.876040            0       True
4     LightGBMClassifierCustom       0.84       0.020979   0.789966                0.020979           0.789966            0       True
5   RandomForestClassifierGini       0.84       0.124382   0.828404                0.124382           0.828404            0     

/usr/local/lib/python3.6/dist-packages/autogluon/utils/plots.py:141: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')
